Ideas:
* there could be a bug in adaptive.hpp
* maybe recursive subdivision is better than gauss-kronrod for this type of problem.
* ~~kahan summation might be necessary. perhaps the adding and subtracting of the error causes problems?~~
* ~~align the python numpy kernels with the nearfield.cpp kernels.
* the problem is in the interpolation!! 

The problem is the interpolation. If I replace the interpolated source locations that suffer from 1 ulp error with analytical exact source locations, I get errors 20x lower. Is this solvable? I'm not sure. I definitely need to interpolate, but perhaps there's a way to do so slightly more accurately? I lean towards thinking this is not possible because there is a fundamental loss of precision due to the derivative in the hypersingular integral.

# Analytic comparison 

Let's use the analytic solution for stress for slip on a line segment in a fullspace extending from y = -1 to y = 1. From page 35 of the Segall book. 

\begin{equation}
\int_{S} K(x, y) y_2 dy
\end{equation}

In [27]:
import mpmath
import numpy as np

def gauss_rule(n):
    mpmath.mp.dps = 30
    points, weights = mpmath.mp.gauss_quadrature(2)
    interp_weights = []
    for i in range(2):
        prod_dist = 1
        for j in range(2):
            if i == j:
                prod_dist *= 1
            else:
                prod_dist *= points[i] - points[j]
        interp_weights.append(1 / prod_dist)
    quad_rule = (
        np.array([float(p) for p in points]),
        np.array([float(w) for w in weights]),
        np.array([float(w) for w in interp_weights]),
    )
    return quad_rule


In [44]:
fx0 = quad_rule[0][0]
fx1 = quad_rule[0][1]
fx01 = (fx1 - fx0) / (quad_rule[0][1] - quad_rule[0][0])
fx0 + fx01 * (x - quad_rule[0][0])

0.9990000000000001

In [56]:
quad_rule = gauss_rule(2)
x = 0.999999
f = (
     x * quad_rule[0][0] * quad_rule[2][0] - quad_rule[0][1] * quad_rule[0][0] * quad_rule[2][0]
    + x * quad_rule[0][1] * quad_rule[2][1] - quad_rule[0][0] * quad_rule[0][1] * quad_rule[2][1] 
) / (
     x * quad_rule[2][0] - quad_rule[0][1] * quad_rule[2][0] 
    + x * quad_rule[2][1] - quad_rule[0][0] * quad_rule[2][1]
)
f, f - x


(0.9999989999999999, -1.1102230246251565e-16)

In [2]:
from tectosaur2.nb_config import setup
setup()

import numpy as np
from tectosaur2 import integrate_term
from tectosaur2.mesh import unit_circle
from tectosaur2.laplace2d import hypersingular
from tectosaur2.global_qbx import global_qbx_self

In [6]:
quad_rule = gauss_rule(2)

import sympy as sp
import matplotlib.pyplot as plt
from tectosaur2 import panelize_symbolic_surface, pts_grid
t = sp.var('t')
fault = panelize_symbolic_surface(t, 0*t, t, quad_rule, n_panels=1)

def analytical_stress(obsx, obsy):
    rp = obsx ** 2 + (obsy + 1) ** 2
    ri = obsx ** 2 + (obsy - 1) ** 2
    sxz = -(1.0 / (2 * np.pi)) * (((obsy + 1) / rp) - ((obsy - 1) / ri))
    syz = (1.0 / (2 * np.pi)) * ((obsx / rp) - (obsx / ri))
    return sxz, syz

from tectosaur2.laplace2d import Hypersingular
ys = np.linspace(1.00005, 1.00005, 1)
xs = [0.0] * ys.shape[0] 
obs_pts = np.array([xs, ys]).T.copy()
sing = np.array([(0,-1), (0, 1)])
stress_mat, report = integrate_term(Hypersingular(d_qbx=0, d_up=1e9), obs_pts, fault, safety_mode=True, singularities=sing, return_report=True)
interior_stress = stress_mat[:,:,:,0].sum(axis=2)
analytical_sxz, analytical_syz = analytical_stress(obs_pts[:,0], obs_pts[:,1])
interior_sxz = interior_stress[:,0]
interior_syz = interior_stress[:,1]
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    sxz_err = np.log10(np.abs(interior_sxz - analytical_sxz))
    syz_err = np.log10(np.abs(interior_syz - analytical_syz))

-0.988703 -0.988703 0
-0.994352 -0.994352 1.11022e-16
0.0056484 0.0056484 2.68882e-17
1 -29.298169478609026 25.158582004556312 -29.298169478609026 25.158582004556312 0
0.066405934403187261 -29.364575413012211
0.0028241993468303261 0.0028241993468303139 1.214306433183765e-17
0.50282419934683031 0.50282419934683031 0
2 -58.597946924577386 49.640522803756397 -58.664352858980571 25.158582004288746 -29.299777445968356
0.037278220567774321 -58.701631079548342
0.50141209967341516 0.50141209967341516 0
0.75141209967341505 0.75141209967341516 -1.1102230246251565e-16
3 -114.56158130938053 95.951048533413513 -114.66526546435149 49.640522802861241 -55.96363438480315
-0.020964102809429227 -114.64430136154206
0.75070604983670752 0.75070604983670752 0
0.87570604983670763 0.87570604983670752 1.1102230246251565e-16
4 -216.58323794440258 178.87864591189367 -216.66595799656412 95.951048531171779 -102.02165663502207
-0.13739635378414855 -216.52856164277998
0.87535302491835387 0.87535302491835376 1.1102230

/Users/tbent/Dropbox/active/eq/tectosaur2/tectosaur2/integrate.py:287: UserWarning: Some integrals failed to converge during adaptive integration. This an indication of a problem in either the integration or the problem formulation.
  warnings.warn(


1817e-10 -169.1604158666826 9.113421128859045e-11 1.3429257705865894e-10
-75.726222546523275 -93.434193320159324
0.99981706690669836 0.99981706690669836 0
0.99984758448482325 0.99984758448482336 -1.1102230246251565e-16
44 -1163.743187141042 6.1841824476811021e-10 -88.613390848204062 8.2751583363460668e-11 6.3309357756224927e-12
-37.630822249613345 -50.982568598590724
0.99997713336333727 0.99997713336333727 0
0.99998094806060311 0.99998094806060289 2.2204460492503131e-16
45 -1163.7431871409817 5.6271047303919127e-10 -93.434193320099041 8.0589757089910563e-11 6.0289551129244501e-11
-44.136857605815486 -49.297335714283548
0.99993900793761226 0.99993900793761215 1.1102230246251565e-16
0.99994663733214328 0.9999466373321434 -1.1102230246251565e-16
46 -1163.743187140969 5.6899999749599525e-10 -83.575323561184661 7.6727735276449494e-11 1.2754242106893798e-11
-38.704145142032644 -44.871178419152024
0.99998476275786863 0.99998476275786852 1.1102230246251565e-16
0.99998857745513425 0.99998857745

In [2]:
report['nearfield_n_subsets']

array([50], dtype=int32)

In [3]:
stress_mat

array([[[[-1163.74318714],
         [ 4346.76247349]],

        [[    0.        ],
         [    0.        ]]]])

In [6]:
correct = analytical_sxz[0]
tct_val = interior_sxz[0]
correct, tct_val

(3183.0192863490306, 3183.019286348184)

In [15]:
from tectosaur2.laplace2d import Hypersingular
from tectosaur2.mesh import barycentric_eval, build_interp_matrix
op = obs_pts
which_basis = np.zeros(fault.panel_order)
which_basis[0] = 1
kernel = Hypersingular()


def integrand(srcy):
    src_normals = np.zeros((srcy.shape[0], 2))
    src_normals[:,0] = -1
    srcyv = srcy#barycentric_eval(srcy, fault.pts[:, 1], quad_rule[2], fault.pts[:, 1])
    entry = kernel.kernel(op, np.array([0*srcy, srcyv]).T.copy(), src_normals)
    phi = barycentric_eval(srcy, fault.pts[:, 1], quad_rule[2], which_basis)
    return entry[0,0,:,0]# * phi

grule = gauss_rule(8)
def simple_quad(domain):
    xs = domain[0] + (domain[1] - domain[0]) * ((grule[0] + 1) * 0.5)
    ws = (domain[1] - domain[0]) * 0.5 * grule[1]
    return integrand(xs).dot(ws)
    
import quadpy
kronrod_n = 6
kronrod_rule = quadpy.c1.gauss_kronrod(kronrod_n)
kronrod_qx = kronrod_rule.points
kronrod_qw = kronrod_rule.weights
gauss_r = quadpy.c1.gauss_legendre(kronrod_n)
gauss_qx = gauss_r.points
kronrod_qw_gauss = gauss_r.weights

import heapq
def gk_quad(domain):
    gxs = domain[0] + (domain[1] - domain[0]) * ((gauss_qx + 1) * 0.5)
    gws = (domain[1] - domain[0]) * 0.5 * kronrod_qw_gauss
    kxs = domain[0] + (domain[1] - domain[0]) * ((kronrod_qx + 1) * 0.5)
    kws = (domain[1] - domain[0]) * 0.5 * kronrod_qw
    vs1 = integrand(gxs) * gws
    vs2 = integrand(kxs) * kws
    est1 = np.sum(vs1)
    est2 = np.sum(vs2)
    # est1 = 0
    # est2 = 0
    # for i in range(vs1.shape[0]):
    #     est1 += vs1[i]
    # for i in range(vs2.shape[0]):
    #     est2 += vs2[i]
    return est1, est2, np.abs(est2 - est1)

def priority_quad(tol):
    low_est, est, err = gk_quad([-1, 1])

    queue = []
    heapq.heappush(queue, (-err, est, -1, 1))

    for i in range(1, 1000):
        cur_integral = heapq.heappop(queue)
        midpt = (cur_integral[2] + cur_integral[3]) * 0.5
        left = gk_quad([cur_integral[2], midpt])
        right = gk_quad([midpt, cur_integral[3]])
        err += cur_integral[0] + left[2] + right[2]

        retro_err = -cur_integral[1] + left[1] + right[1]
        est += -cur_integral[1] + left[1] + right[1]
        heapq.heappush(queue, (-left[2], left[1], cur_integral[2], midpt))
        heapq.heappush(queue, (-right[2], right[1], midpt, cur_integral[3]))
        print(i, est, err, left[1] + right[1], cur_integral[0], retro_err)
        print(left[1], right[1])
        if err < tol:
            break
    return est, i, err

val, n_integrals, err = priority_quad(1e-12)
val, val - correct, val - tct_val, n_integrals, err

1 82.82947691373101 69.41690774361035 82.829476913731 -35.03323524179466 41.12074828905089
0.07957150358371452 82.74990541014729
2 163.11974336288188 136.28676663992593 163.04017185929817 -69.416907742626 80.29026644915088
0.15913107263534468 162.88104078666282
3 316.2391303463868 262.77868101876186 316.0004277701678 -136.28676663697382 153.11938698350494
0.31821441549485385 315.6822133546729
4 595.1592013796819 489.28429848731497 594.6022843879681 -262.7786810118782 278.92007103329513
0.6362379786813337 593.9660464092867
5 1060.8713452392617 853.4294351189542 1059.6781902688663 -489.28429847258377 465.71214385957967
1.2717130820884672 1058.406477186778
6 1724.636170798553 1326.3521000521907 1722.1713027460692 -853.4294350885903 663.7648255592912
2.5403789283572276 1719.6309238177118
7 2447.431547461546 1711.2271185231446 2442.426300480705 -1326.3520999908087 722.7953766629932
5.068602933612242 2437.357697547093
8 2963.609407206653 1691.7385075967775 2953.5355572922 -1711.2271184007031

(3183.0192863490265,
 -4.092726157978177e-12,
 8.42646841192618e-10,
 861,
 9.996135863499234e-13)

In [44]:
def integrand(srcy):
    src_normals = np.zeros((srcy.shape[0], 2))
    src_normals[:,0] = -1
    srcyv = barycentric_eval(srcy, fault.pts[:, 1], quad_rule[2], fault.pts[:, 1])
    entry = kernel.kernel(op, np.array([0*srcy, srcyv]).T.copy(), src_normals)
    phi = barycentric_eval(srcy, fault.pts[:, 1], quad_rule[2], which_basis)
    return entry[0,0,:,0]# * phi

grule = gauss_rule(10)
def simple_quad(domain):
    xs = domain[0] + (domain[1] - domain[0]) * ((grule[0] + 1) * 0.5)
    ws = (domain[1] - domain[0]) * 0.5 * grule[1]
    return np.sum(integrand(xs) * ws)

def recursive_quad(domain, tol, base_value=None):
    if base_value is None:
        base_value = simple_quad(domain)
    center = (domain[0] + domain[1]) * 0.5
    ldom = [domain[0], center]
    lval = simple_quad(ldom)

    rdom = [center, domain[1]]
    rval = simple_quad(rdom)
    better_value = lval + rval
    err = np.abs(better_value - base_value)
    if err < tol:
        return better_value, 2, err
    else:
        left = recursive_quad(ldom, tol, lval)
        right = recursive_quad(rdom, tol, rval)
        return left[0] + right[0], left[1] + right[1], left[2] + right[2]

val, n_integrals, err = recursive_quad([-1, 1], 1e-12)
val, val - correct, (val - correct) / correct, val - tct_val, n_integrals, err

(3183.0192863490815,
 5.093170329928398e-11,
 1.6001066508680627e-14,
 8.976712706498802e-10,
 3546,
 7.628677828330765e-10)